In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import numpy as np


# Sample Text

text = "I love deep learning and I love pytorch"

# Tokenize
tokens = text.lower().split()
vocab = {word: idx for idx, word in enumerate(set(tokens))}
rev_vocab = {idx: word for word, idx in vocab.items()}
vocab_size = len(vocab)

# Convert to indices
indices = [vocab[w] for w in tokens]

# Prepare Dataset
seq_len = 2   
X, y = [], []

for i in range(len(indices) - seq_len):
    X.append(indices[i:i+seq_len])   
    y.append(indices[i+seq_len])     

X = torch.tensor(X)  # shape: (batch, seq_len)
y = torch.tensor(y)  # shape: (batch,)

# Define RNN Model
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embed(x)                      # (batch, seq_len, embed_dim)
        out, hidden = self.rnn(x)              # out: (batch, seq_len, hidden_dim)
        out = out[:, -1, :]                   
        out = self.fc(out)                     # (batch, vocab_size)
        return out

# Train Model
model = RNNModel(vocab_size, embed_dim=8, hidden_dim=16)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Predict Next Word
def predict_next(seq):
    model.eval()
    seq_idx = torch.tensor([[vocab[w] for w in seq.split()]])
    with torch.no_grad():
        out = model(seq_idx)
        pred_idx = torch.argmax(out, dim=1).item()
    return rev_vocab[pred_idx]

print("\nPrediction:")
print("Input: 'i love' -> Next word:", predict_next("i love"))
print("Input: 'love deep' -> Next word:", predict_next("love deep"))


Epoch 50, Loss: 0.2427
Epoch 100, Loss: 0.2356
Epoch 150, Loss: 0.2339
Epoch 200, Loss: 0.2330

Prediction:
Input: 'i love' -> Next word: deep
Input: 'love deep' -> Next word: learning


In [3]:
import torch
import torch.nn as nn
import numpy as np


text="My name is Aasif. I am going to be next president of my kingdom. Pray for me"


token=text.lower().split()
token
type(token)
type(set(token))

unique={}
rev_unique={}
for idx, word in enumerate(set(token)):
    unique[word]=idx
    rev_unique[idx]=word
    
unique
rev_unique

indices=[]
for i in token:
    indices.append(unique[i])
indices

#make dataset

seq_len=2

x=[]
y=[]

for i in range(len(indices)-seq_len):
    x.append(indices[i:i+seq_len])
    y.append(indices[i+seq_len])
x
y

x=torch.tensor(x)
y=torch.tensor(y)
x
y







tensor([ 4, 15,  1,  7, 10, 13, 11,  5, 12,  8,  9,  0,  3,  2,  6])

In [4]:
unique
len(indices)

17

In [5]:
import torch
import torch.nn as nn

text="I love cricket very much"
token=text.lower().split()
vocab={j:i for i, j in enumerate(set(token))}
rev_vocab={i:j for i, j in enumerate(set(token))}
unique_total_word=len(set(token))
unique_total_word

rev_vocab
embed_dim=8
embedding=nn.Embedding(len(vocab),embed_dim)

sample_input=torch.tensor([2,3,1]) # cricket, very, love

embed_vector=embedding(sample_input)
embed_vector

tensor([[-0.2509,  0.5263, -1.0453, -0.7562,  0.7853, -0.2135, -1.1515, -0.6225],
        [ 0.4355, -0.2811, -1.3441,  1.4013,  1.2535,  0.2069,  0.9420, -0.7660],
        [ 1.1204,  0.9889,  0.2480, -1.1046, -2.6616, -0.5193,  0.5407,  0.8385]],
       grad_fn=<EmbeddingBackward0>)

In [6]:
vocab

{'i': 0, 'love': 1, 'much': 2, 'cricket': 3, 'very': 4}

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# -------------------------
# 1. Data Preparation (Corrected)
# -------------------------
text="""Sheikh Hasina Wazed (born September 28, 1947) is a prominent Bangladeshi politician and the daughter of the nation's founding father, Sheikh Mujibur Rahman. She served two separate tenures as the tenth Prime Minister of Bangladesh, first from June 1996 to July 2001, and then for four consecutive terms from January 2009 until her resignation in August 2024, making her the longest-serving head of government in the country's history. Hasina's political career began after surviving the 1975 assassination of her family and returning from exile in 1981 to lead the Awami League political party, spearheading the movement to restore democracy. Her later and longest premiership was credited with overseeing significant economic growth, substantial poverty reduction, and massive infrastructure development (such as the Padma Bridge and Dhaka Metro Rail), alongside the implementation of the "Digital Bangladesh" vision. However, her final years in power were marred by increasing international and domestic criticism over allegations of authoritarianism, the erosion of democratic institutions, and suppression of political dissent and human rights abuses, ultimately leading to widespread student-led protests that forced her resignation and flight from the country in August 2024."""
tokens = text.lower().split()
vocab = {word: idx for idx, word in enumerate(set(tokens))}
rev_vocab = {idx: word for word, idx in vocab.items()}

indices = [vocab[w] for w in tokens]

# dataset বানাই: sequence -> next word
seq_len = 2
X, y = [], []
for i in range(len(indices) - seq_len):
    X.append(indices[i:i+seq_len])   # input sequence
    y.append(indices[i+seq_len])     # target word

# --- FIX APPLIED HERE ---
# X and y must be converted to torch.long (integer type) for nn.Embedding
X = torch.tensor(X).long()   # shape: (samples, seq_len)
y = torch.tensor(y).long()   # shape: (samples,)
# ------------------------

# -------------------------
# 2. RNN Model
# -------------------------
class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embed(x)                         # [batch, seq_len, embed_dim]
        out, h = self.rnn(x)                      # [batch, seq_len, hidden_dim]
        out = out[:, -1, :]                       # last timestep only
        out = self.fc(out)                        # [batch, vocab_size]
        return out

# -------------------------
# 3. Training
# -------------------------
vocab_size = len(vocab)
model = RNNPredictor(vocab_size, embed_dim=50, hidden_dim=50)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 200
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_fn(output, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss = {loss.item():.4f}")

# -------------------------
# 4. Prediction Function (Minor change to handle potential unknown words)
# -------------------------
def predict_next(model, text_seq):
    model.eval()
    tokens = text_seq.lower().split()
    
    # Only use words that are in the vocab
    idx_seq = [vocab[w] for w in tokens if w in vocab]
    
    # Check if the sequence length matches the model's seq_len
    if len(idx_seq) < seq_len:
        print(f"Error: Input sequence needs at least {seq_len} known words.")
        return "N/A"
        
    idx_seq = idx_seq[-seq_len:] # Use only the last seq_len tokens
    
    # --- FIX APPLIED HERE ---
    # The input tensor must also be cast to torch.long
    idx_seq = torch.tensor(idx_seq).unsqueeze(0).long()  # [1, seq_len], ensuring long type
    # ------------------------
    
    with torch.no_grad():
        out = model(idx_seq)
        pred_idx = torch.argmax(out, dim=1).item()
    return rev_vocab[pred_idx]


test_seq = "substantial poverty" 
pred_word = predict_next(model, test_seq)
print(f"\nInput: {test_seq} → Predicted next word: {pred_word}")

Epoch 50, Loss = 0.0401
Epoch 100, Loss = 0.0341
Epoch 150, Loss = 0.0329
Epoch 200, Loss = 0.0321

Input: substantial poverty → Predicted next word: reduction,
